In [5]:
from itertools import count
import torch
import torch.nn as nn
import torch.nn.functional as F

class Model(nn.Module):
    def __init__(self, in_features=4, H1=8, H2=9, out_features=3):
        super(Model, self).__init__()  # Correcting super() call indentation
        self.fc1 = nn.Linear(in_features, H1)  # Correcting capitalization to nn.Linear
        self.fc2 = nn.Linear(H1, H2)  # Correcting capitalization to nn.Linear
        self.out = nn.Linear(H2, out_features)  # Correcting capitalization to nn.Linear

    def forward(self, x):
        x = F.relu(self.fc1(x))  # Correcting indentation
        x = F.relu(self.fc2(x))  # Correcting indentation
        x = self.out(x)
        return x

# Pick a manual seed for randomization
torch.manual_seed(21)
model = Model()

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

url = 'https://gist.githubusercontent.com/curran/a08a1080b88344b0c8a7/raw/0e7a9b0a5d22642a06d3d5b9bcbad9890c8ee534/iris.csv'
my_df = pd.read_csv(url)

#Changed the last column from strings to int
my_df['species'] = my_df['species'].replace('setosa', 0)
my_df['species'] = my_df['species'].replace('versicolor', 1)
my_df['species'] = my_df['species'].replace('virginica', 2)

#Train, test, split! Set a,b  Variables that will store the split values
a = my_df.drop('species', axis=1)          #We are training our model to predict species on the basis on 4 features
b = my_df['species']                       # Species--> Output variable; so we have split the dataset as input/output variables

#Convert these into numpy arrays
a = a.values
b = b.values

from sklearn.model_selection import train_test_split
#Train Test Split
a_train, a_test, b_train, b_test = train_test_split(a,b, test_size=0.2, random_state=21)
#Test size 20%-->0.2, so Pytorch knows that the rest, training size, is 80%

#Convert a features to float tensors
a_train = torch.FloatTensor(a_train)  # Float bcz the numbers in the array are decimals
a_test = torch.FloatTensor(a_test)

#Convert b labels to float tensors long
b_train = torch.LongTensor(b_train)
b_test = torch.LongTensor(b_test)
# Long tensors are 64bit integers

# Set the criterion of model to measure the error
# How far off the predictions are from the data
criterion = nn.CrossEntropyLoss()
import torch.optim as optim
# Choose Adam Optimizer
# lr = Learning rate (if error doesn't go down after a bunch of iterations (epochs), lower the learning rate)
optimizer= optim.Adam(model.parameters(),lr=0.01)        #lower the lr, the longer it takes for the model to learn

# Set the criterion of model to measure the error
# How far off the predictions are from the data
criterion = nn.CrossEntropyLoss()
import torch.optim as optim
# Choose Adam Optimizer
# lr = Learning rate (if error doesn't go down after a bunch of iterations (epochs), lower the learning rate)
optimizer= optim.Adam(model.parameters(),lr=0.1)        #lower the lr, the longer it takes for the model to learn

#Train our model
epochs = 100
losses = []
for i in range(epochs):
  b_pred = model.forward(a_train)
  loss = criterion(b_pred, b_train)
  losses.append(loss.detach().numpy())       # loss would be a tensor, convert as numpy
  if i % 10 == 0:
    print (f'Epoch {i} and loss: {loss}')

  # Do backpropagation: Take the error rate of forward propagation
  # Feed it back through the network to fine-tune the weights
  optimizer.zero_grad()  # Zero the gradients before running the backward pass
  loss.backward()  # Backpropagation: Compute gradients of the loss with respect to model parameters
  optimizer.step()  # Perform a single optimization step (parameter update)



#Create graph
 #plt.plot(range(epochs), losses)
 #plt.ylabel("loss/error")
 #plt.xlabel('Epoch')

#Evaluate Model on Test dataset (Model Validation)
with torch.no_grad():                     #turn off back propagation
  b_eval = model.forward(a_test)          #a_test are features from our test set
                                          # y_eval are predictions
  loss = criterion(b_eval, b_test)        # Find loss: prediction vs test set value

# The diff of the above loss and loss in the training data we found above is high
correct = 0
with torch.no_grad():
  for i, data in enumerate(a_test):
    b_val = model.forward(data)  # Will tell what flower type our NN think it is
    print( f'{i+1}.) {str(b_val)} \t {b_test[i]} \t {b_val.argmax().item()}')
    # Prediction is correct or not
    if b_val.argmax().item() == b_test[i]:
      correct +=1

print(f'We got {correct} correct')
print(f' Fuck off _|_ \t My first Neural Network is here')
# In the output tensor, we are getting 3 values bcz our out has 3 nodes
# As there are 3 species of Iris flower. The model is evaluating which out of 3 is correct
# The correct species is reflected by max weigth of a node.


#Passing a new iris to see what the model predicts
 #new_iris = torch.tensor([1.45, 5.65, 3.54, 2.4])
 #with torch.no_grad():
 #  print (model(new_iris))

# Save our NN Model in the trained state into a dictionary (where weights of the nodes are adjusted from the training)
torch.save(model.state_dict(), 'My_Iris_NN.pt')

#Load the saved model
new_model = Model()
new_model.load_state_dict(torch.load('My_Iris_NN.pt'))

#Evaluate the model
new_model.eval()

Epoch 0 and loss: 1.1224557161331177
Epoch 10 and loss: 0.26079532504081726
Epoch 20 and loss: 0.11196835339069366
Epoch 30 and loss: 0.0652126744389534
Epoch 40 and loss: 0.045915890485048294
Epoch 50 and loss: 0.030438585206866264
Epoch 60 and loss: 0.0274417195469141
Epoch 70 and loss: 0.025059852749109268
Epoch 80 and loss: 0.023842932656407356
Epoch 90 and loss: 0.022803129628300667
1.) tensor([-3.4486,  7.8776, -7.0923]) 	 1 	 1
2.) tensor([ 20.5991,  10.0827, -32.0010]) 	 0 	 0
3.) tensor([ 21.0665,  10.2620, -32.6718]) 	 0 	 0
4.) tensor([ 21.5690,  10.4547, -33.3928]) 	 0 	 0
5.) tensor([-3.4142,  7.8415, -7.0920]) 	 1 	 1
6.) tensor([-4.2982,  8.5767, -7.4055]) 	 1 	 1
7.) tensor([ 21.4622,  10.4138, -33.2396]) 	 0 	 0
8.) tensor([-20.4106,   1.2757,  11.1640]) 	 2 	 2
9.) tensor([ 20.0085,   9.9837, -31.2776]) 	 0 	 0
10.) tensor([ 22.1648,  10.6833, -34.2479]) 	 0 	 0
11.) tensor([-0.7237,  8.6255, -9.6157]) 	 1 	 1
12.) tensor([-3.9495,  7.5978, -6.4779]) 	 1 	 1
13.) tens

Model(
  (fc1): Linear(in_features=4, out_features=8, bias=True)
  (fc2): Linear(in_features=8, out_features=9, bias=True)
  (out): Linear(in_features=9, out_features=3, bias=True)
)

In [ ]:
torch.save(model.state_dict(), 'Iris flower NN model')


In [ ]:
new_model = model()
new.model.load_state_dict(torch.load('Iris flower NN model'))

TypeError: Model.forward() missing 1 required positional argument: 'x'

In [3]:
new_model.eval()

Model(
  (fc1): Linear(in_features=4, out_features=8, bias=True)
  (fc2): Linear(in_features=8, out_features=9, bias=True)
  (out): Linear(in_features=9, out_features=3, bias=True)
)